In [1]:
import argparse
import torch
import os
import json
from tqdm import tqdm
import shortuuid
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import math
import torchvision.transforms as T
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer
import sys
import re
import gc
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg


IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)


def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform


def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio


def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images


def load_image(image_file, input_size=448, max_num=12, use_dynamic=True):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    if use_dynamic:
        images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
        print("Use Dynamic")
    else:
        images = [image]
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values


def split_list(lst, n):
    """Split a list into n (roughly) equal-sized chunks"""
    chunk_size = math.ceil(len(lst) / n)  # integer division
    return [lst[i:i+chunk_size] for i in range(0, len(lst), chunk_size)]


def get_chunk(lst, n, k):
    chunks = split_list(lst, n)
    return chunks[k]


def load_json_jsonl(file_path):
    # Check the file extension and process accordingly
    if file_path.endswith(".jsonl"):
        # For JSONL files (line-delimited JSON)
        data = [json.loads(line) for line in open(file_path, "r")]
    elif file_path.endswith(".json"):
        # For standard JSON files
        with open(file_path, "r") as f:
            data = json.load(f)  # Load the entire JSON file as a single object
    else:
        raise ValueError(f"Unsupported file format: {file_path}")
    return data


def show_image_with_bbox(image_path, bboxes):
    """
    Display an image with bounding boxes in Jupyter Notebook at its original size.

    Args:
        image_path (str): Path to the image file.
        bboxes (list): List of bounding boxes, each represented as (x1, y1, x2, y2).
    """
    # 读取图片
    img = mpimg.imread(image_path)
    
    # 获取图像的原始尺寸
    img_height, img_width = img.shape[:2]
    
    # 创建图形和轴，设置图形大小为图像的原始尺寸
    dpi = 80  # 可以根据需要调整 DPI
    figsize = (img_width / dpi, img_height / dpi)
    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    
    # 显示图片
    ax.imshow(img)

    if bboxes is not None:
        # 绘制每个边界框
        for bbox in bboxes:
            x1, y1, x2, y2 = bbox
            x1 = int(x1 / 1000 * img_width)
            y1 = int(y1 / 1000 * img_height)
            x2 = int(x2 / 1000 * img_width)
            y2 = int(y2 / 1000 * img_height)
            rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=5, edgecolor='blue', facecolor='none')
            ax.add_patch(rect)
    
    # 移除轴标记
    ax.axis('off')
    
    # 显示图形
    plt.show()


def extract_characters_regex(s, choices):
    s = s.strip()
    answer_prefixes = [
        "The best answer is",
        "The correct answer is",
        "The answer is",
        "The answer",
        "The best option is"
        "The correct option is",
        "Best answer:"
        "Best option:",
    ]
    for answer_prefix in answer_prefixes:
        s = s.replace(answer_prefix, "")

    if len(s.split()) > 10 and not re.search("[ABCDE]", s):
        return ""
    matches = re.search(r'[ABCDE]', s)
    if matches is None:
        for choice in choices:
            if s.lower() in choice.lower():
                return choice[1]
        return ""
    return matches[0]

/home/zilun/anaconda3/envs/imagerag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
baseline_answer_fpath = "/media/zilun/fanxiang4t/GRSM/ImageRAG_git/codebase/inference/MME-RealWorld-RS/answer_baseline-mme-lite_8B_detection-gt.jsonl"
imagerag_answer_fpath = "/media/zilun/fanxiang4t/GRSM/ImageRAG_git/codebase/inference/MME-RealWorld-RS/answer-corrected_zoom4klora32-mme-lite_8B_detection-gt-baseline-cot.jsonl"
image_root = "/media/zilun/wd-161/datasets/MME-RealWorld/rs_subset/remote_sensing"

In [9]:
baseline_answers = load_json_jsonl(baseline_answer_fpath)
imagerag_answers = load_json_jsonl(imagerag_answer_fpath)
print(len(baseline_answers), len(imagerag_answers))
imagerag_answers[0]

150 150


{'Question_id': 'perception/remote_sensing/position/2527',
 'Image': 'dota_v2_dota_v2_dota_v2_P7036.png',
 'Text': 'Where is the yellow crane in the picture?',
 'Question Type': 'Multiple Choice',
 'Answer choices': ['(A) In the upper right area of the picture',
  '(B) In the upper left area of the picture',
  '(C) In the lower right area of the picture',
  '(D) In the lower left area of the picture',
  "(E) This image doesn't feature the position."],
 'Ground truth': 'A',
 'Category': 'position',
 'Subtask': 'Remote Sensing',
 'Task': 'Perception',
 'gt_toi': [6592, 970, 6970, 1569],
 'img_size': [7360, 4912],
 'output': "To determine the position of the yellow crane in the picture, we need to carefully examine the aerial view of the city. The crane is typically a tall structure, often used in construction, and is usually quite distinct due to its color and shape.\n\n1. **Upper Right Area (A)**: This area contains several large buildings, but there is no visible yellow crane.\n2. **Up

In [10]:
print(os.getcwd())

/media/zilun/fanxiang4t/GRSM/ImageRAG_git/codebase/dataset_process


In [17]:
def prepare_input(image_root, line, mode="baseline"):
    # print(line)
    test_prompt = "Select the best answer to the above multiple-choice question based on the image. Respond with only the letter (A, B, C, D, or E) of the correct option."
    image_name = line["Image"]
    image_path = os.path.join(image_root, image_name)
    image = Image.open(image_path).convert('RGB')
    w, h = image.size 
    choices = line['Answer choices']
    qs = line["Text"]
    choice_prompt = ' The choices are listed below: \n'
    for choice in choices:
        choice_prompt += choice + "\n"
    qs += choice_prompt + test_prompt + '\nThe best answer is:'
    output = line["output"]
    # 'Ground truth': 'A', 'Category': 'position'
    gt = line["Ground truth"]
    category = line["Category"]
    
    if mode == "imagerag":
        try:
            predict_bbox = line["visual_cue"]
            predict_bbox = [int(predict_bbox[0]) / w * 1000, int(predict_bbox[1]) / h * 1000, int(predict_bbox[2]) / w * 1000, int(predict_bbox[3]) / h * 1000]
            qs = line["final_instruction"]
        except:
            predict_bbox = [0, 0, 1000, 1000]
        print(predict_bbox)
        
        # toi = line["target_of_interest"]
        return image_name, image_path, qs, output, gt, category, [predict_bbox]
    else:
        return image_name, image_path, qs, output, gt, category
        

In [18]:
start = 0
end = start + 20

In [20]:
# for i in range(start, end):
#     baseline_answer = baseline_answers[i]
#     imagerag_answer = imagerag_answers[i]
#     baseline_img_name, baseline_img_path, baseline_qs, baseline_output, baseline_gt, baseline_category = prepare_input(image_root, baseline_answer, mode="baseline")
#     imagerag_img_name, imagerag_img_path, imagerag_qs, imagerag_output, imagerag_gt, imagerag_category, imgrag_boxes = prepare_input(image_root, imagerag_answer, mode="imagerag")
#     assert baseline_img_name == imagerag_img_name
#     # assert baseline_qs == imagerag_qs
#     # imgrag_boxes = [[20.0, 949.1666666666667, 39.16666666666667, 999.6666666666667]]
#     show_image_with_bbox(imagerag_img_path, imgrag_boxes)
#     print("Index: {}\nQuestion: {}\nCategory: {}\nGT: {}\nBaseline Output: {}\nImageRAG Output: {}\nImageRAG Bbox: {}\n".format(i+1, imagerag_qs, imagerag_category, imagerag_gt, baseline_output, imagerag_output, imgrag_boxes))